- Setup the environment

In [1]:
import sys
import os
cur_dir = os.getcwd()
# Add the current directory to system path
sys.path.append(cur_dir)

- Load input data

Create sample data

In [2]:
import pandas as pd
import numpy as np
# Prepare sample data
timestamp = pd.date_range(start='2020-01-01', periods=300, freq='D')
time_series = pd.DataFrame({'values': np.random.randn(300)}, index=timestamp)
labels = pd.DataFrame({'label': np.random.randint(0, 3, size=300)}, index=timestamp)
ext_features = pd.DataFrame({
    'feature1': np.random.randn(300),
    'feature2': np.random.randn(300)
}, index=timestamp)

combined_data = time_series.join(labels).join(ext_features)

Train / Validation / Test split

In [3]:
# Train / Validation / Test Split
train_data = combined_data[:100]
val_data = combined_data[100:200]
test_data = combined_data[200:]

# Read targets
train_target = train_data.label.values
val_target = val_data.label.values
test_target = test_data.label.values

# Read features
train_features = train_data[['feature1', 'feature2']].values
val_features = val_data[['feature1', 'feature2']].values
test_features = test_data[['feature1', 'feature2']].values

In [4]:
from pl_model_utils import TimeSeriesDataModule
# Instantiate data module and model
data_module = TimeSeriesDataModule(
    train_target, train_features,
    val_target, val_features,
    test_target, test_features,
    batch_size=16
)

# Setup the data for model
data_module.setup()

c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Classification with Basic Autoencoder

- Compile simple Autoencoder for Classification

In [5]:
from pl_autoencoder_classifiers import AutoencoderClassifier
ae_model = AutoencoderClassifier(context_length=1, num_classes=3, num_features=2)

In [6]:
from pytorch_lightning.callbacks import ModelCheckpoint
# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='train_loss',
    mode='min',
    save_top_k=1,
    save_last=True,
)

- Train the Autoencoder

In [7]:
from pytorch_lightning import Trainer
# Train the model
trainer = Trainer(max_epochs=20, callbacks=[checkpoint_callback])
trainer.fit(ae_model, data_module.train_dataloader())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type       | Params
------------------------------------------
0 | encoder    | Sequential | 8.8 K 
1 | decoder    | Sequential | 8.4 K 
2 | classifier | Sequential | 2.2 K 
3 | softmax    | Softmax    | 0     
------------------------------------------
19.4 K    Trainable params
0         Non-trainable params
19.4 K    Total params
0.078     Total estimated model params size (MB)
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\jedre\miniconda3\envs\GluonTS\lib\si

Epoch 19: 100%|██████████| 7/7 [00:00<00:00, 80.14it/s, v_num=41] 

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 7/7 [00:00<00:00, 62.01it/s, v_num=41]


- Cross-Validation

In [8]:
from pl_model_utils import cross_validate_model
# Perform cross-validation
cv_results = cross_validate_model(data_module.val_features, data_module.val_target, AutoencoderClassifier, context_length=1, num_classes=3, num_features=2)

TypeError: __init__() got an unexpected keyword argument 'num_heads'

- Predictions with MC Dropout Uncertainty

In [17]:
from pl_model_utils import mc_dropout_predictions
# Perform MC Dropout predictions
mc_predictions = mc_dropout_predictions(ae_model, data_module.test_dataloader())
# Calculate mean and standard deviation for uncertainty estimates
mean_predictions = mc_predictions.mean(axis=0)
std_predictions = mc_predictions.std(axis=0)
# Convert mean predictions to class labels
predicted_labels = np.argmax(mean_predictions, axis=1)

## Classification with Attention-Enchanced Autoencoder

## Hyperparameter Tuning

In [9]:
from pytorch_lightning.callbacks import EarlyStopping
from pl_autoencoder_classifiers import AutoencoderAttentionClassifier
from pl_model_utils import TimeSeriesDataset
import pytorch_lightning as pl
import torch
from sklearn.model_selection import TimeSeriesSplit
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import log_loss
import optuna
import numpy as np

def ae_attention_objective(trial):
    context_length = 1
    num_classes = 3
    num_features = 2

    # Suggest hyperparameters
    lr = trial.suggest_categorical('lr', [1e-5, 1e-3, 1e-2])
    num_heads = trial.suggest_categorical('num_heads', [1, 2, 4])
    dropout_prob = trial.suggest_categorical('dropout_prob', [0.1, 0.3, 0.5])
    hidden_units = trial.suggest_categorical('hidden_units', [64, 128, 256])
    embed_dim = trial.suggest_categorical('embed_dim', [32, 64, 128])
    classifier_units = trial.suggest_categorical('classifier_units', [16, 32, 64])
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    # Initialize the model with suggested hyperparameters
    model = AutoencoderAttentionClassifier(
        context_length=context_length,
        num_classes=num_classes,
        num_features=num_features,
        num_heads=num_heads,
        dropout_prob=dropout_prob,
        hidden_units=hidden_units,
        embed_dim=embed_dim,
        classifier_units=classifier_units,
        lr=lr
    )

    # Assuming you have your dataset in `X` and `y`
    X, y = data_module.val_features, data_module.val_target
    # X = np.array(X_train)  # Ensure X_train is a NumPy array
    # y = np.array(y_train)  # Ensure y_train is a NumPy array

    # Time series split
    tscv = TimeSeriesSplit(n_splits=5)
    cv_scores = []

    for train_index, val_index in tscv.split(X):
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]

        # Create DataLoader for the training and validation fold
        train_dataset = TimeSeriesDataset(
            torch.tensor(y_train_fold, dtype=torch.float32), 
            torch.tensor(X_train_fold, dtype=torch.float32)
            )
        
        val_dataset = TimeSeriesDataset(
            torch.tensor(y_val_fold, dtype=torch.float32), 
            torch.tensor(X_val_fold, dtype=torch.float32)
            )
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        # Initialize trainer
        trainer = pl.Trainer(
            max_epochs=10,
            callbacks=[EarlyStopping(monitor='train_loss', patience=3, mode='min')],
            logger=False,
            enable_checkpointing=False
        )

        # Train the model
        trainer.fit(model, train_loader)

        # Validate the model

        model.eval()
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for batch in val_loader:
                targets, features = batch
                _, classification = model(targets, features)
                # preds = torch.argmax(classification, dim=1)
                all_preds.extend(classification.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())

        # val_predictions = trainer.predict(model, val_loader)
        # val_predictions = torch.cat([x for x in val_predictions], dim=0).numpy()
        
        val_loss = log_loss(all_targets, all_preds)
        cv_scores.append(val_loss)

    return np.mean(cv_scores)

In [10]:
# Run the Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(ae_attention_objective, n_trials=100)

[I 2024-07-17 17:46:29,096] A new study created in memory with name: no-name-df67b1ea-18ca-4025-a4de-62baeb978f75
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params
--------------------------------------------------
0 | encoder    | Sequential         | 17.5 K
1 | attention  | MultiheadAttention | 16.6 K
2 | decoder    | Sequential         | 16.9 K
3 | classifier | Sequential         | 1.1 K 
4 | softmax    | Softmax            | 0     
--------------------------------------------------
52.1 K    Trainable params
0         Non-trainable params
52.1 K    Total params
0.208     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 1/1 [00:00<00:00, 65.25it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 1/1 [00:00<00:00, 54.47it/s]


c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\sklearn\metrics\_classification.py:2942: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params
--------------------------------------------------
0 | encoder    | Sequential         | 17.5 K
1 | attention  | MultiheadAttention | 16.6 K
2 | decoder    | Sequential         | 16.9 K
3 | classifier | Sequential         | 1.1 K 
4 | softmax    | Softmax            | 0     
--------------------------------------------------
52.1 K    Trainable params
0         Non-trainable params
52.1 K    Total params
0.208     Total estimated model params size (MB)
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader

...

c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\sklearn\metrics\_classification.py:2942: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params
--------------------------------------------------
0 | encoder    | Sequential         | 33.9 K
1 | attention  | MultiheadAttention | 66.0 K
2 | decoder    | Sequential         | 33.3 K
3 | classifier | Sequential         | 8.5 K 
4 | softmax    | Softmax            | 0     
--------------------------------------------------
141 K     Trainable params
0         Non-trainable params
141 K     Total params
0.567     Total estimated model params size (MB)
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader

Epoch 6: 100%|██████████| 5/5 [00:00<00:00, 40.24it/s]


c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\sklearn\metrics\_classification.py:2942: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params
--------------------------------------------------
0 | encoder    | Sequential         | 33.9 K
1 | attention  | MultiheadAttention | 66.0 K
2 | decoder    | Sequential         | 33.3 K
3 | classifier | Sequential         | 8.5 K 
4 | softmax    | Softmax            | 0     
--------------------------------------------------
141 K     Trainable params
0         Non-trainable params
141 K     Total params
0.567     Total estimated model params size (MB)
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader

Epoch 8: 100%|██████████| 6/6 [00:00<00:00, 40.77it/s]


c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\sklearn\metrics\_classification.py:2942: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
[I 2024-07-17 17:51:53,931] Trial 99 finished with value: 0.13487873163124273 and parameters: {'lr': 0.001, 'num_heads': 1, 'dropout_prob': 0.1, 'hidden_units': 256, 'embed_dim': 128, 'classifier_units': 64, 'batch_size': 16}. Best is trial 81 with value: 0.0960632309753097.


In [11]:
# Print best hyperparameters
print("Best hyperparameters:", study.best_params)

Best hyperparameters: {'lr': 0.001, 'num_heads': 1, 'dropout_prob': 0.1, 'hidden_units': 256, 'embed_dim': 128, 'classifier_units': 64, 'batch_size': 16}


In [12]:
ae_attention_model = AutoencoderAttentionClassifier(
    context_length=1, 
    num_classes=3, 
    num_features=2,
    lr=study.best_params['lr'],
    num_heads=study.best_params['num_heads'],
    dropout_prob=study.best_params['dropout_prob'],
    hidden_units=study.best_params['hidden_units'],
    embed_dim=study.best_params['embed_dim'],
    classifier_units=study.best_params['classifier_units']
    )

In [13]:
# Train the model
trainer = Trainer(max_epochs=20, callbacks=[checkpoint_callback])
trainer.fit(ae_attention_model, data_module.train_dataloader())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:612: UserWarning: Checkpoint directory c:\Users\jedre\OneDrive\Pulpit\State Street\Programs\Python\ML Research Project 2024\Autoencoder\lightning_logs\version_41\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name       | Type               | Params
--------------------------------------------------
0 | encoder    | Sequential         | 33.9 K
1 | attention  | MultiheadAttention | 66.0 K
2 | decoder    | Sequential         | 33.3 K
3 | classifier | Sequential         | 8.5 K 
4 | softmax    | Softmax            | 0     
--------------------------------------------------
141 K     Trainable params
0         Non-trainable params
141 K     Total params
0.567     Total es

Epoch 19: 100%|██████████| 7/7 [00:00<00:00, 64.85it/s, v_num=42]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 7/7 [00:00<00:00, 54.91it/s, v_num=42]


In [14]:
# Perform cross-validation
cv_results = cross_validate_model(
    data_module.val_features, 
    data_module.val_target, 
    AutoencoderAttentionClassifier, 
    context_length=1, 
    num_classes=3, 
    num_features=2,
    lr=study.best_params['lr'],
    num_heads=study.best_params['num_heads'],
    dropout_prob=study.best_params['dropout_prob'],
    hidden_units=study.best_params['hidden_units'],
    embed_dim=study.best_params['embed_dim'],
    classifier_units=study.best_params['classifier_units'])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params
--------------------------------------------------
0 | encoder    | Sequential         | 33.9 K
1 | attention  | MultiheadAttention | 66.0 K
2 | decoder    | Sequential         | 33.3 K
3 | classifier | Sequential         | 8.5 K 
4 | softmax    | Softmax            | 0     
--------------------------------------------------
141 K     Trainable params
0         Non-trainable params
141 K     Total params
0.567     Total estimated model params size (MB)
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\loops\fit_loop.py:280: PossibleUserWarning: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 49: 100%|██████████| 2/2 [00:00<00:00, 47.17it/s, v_num=43]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 2/2 [00:00<00:00, 30.31it/s, v_num=43]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params
--------------------------------------------------
0 | encoder    | Sequential         | 33.9 K
1 | attention  | MultiheadAttention | 66.0 K
2 | decoder    | Sequential         | 33.3 K
3 | classifier | Sequential         | 8.5 K 
4 | softmax    | Softmax            | 0     
--------------------------------------------------
141 K     Trainable params
0         Non-trainable params
141 K     Total params
0.567     Total estimated model params size (MB)
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `Dat

...

Epoch 49: 100%|██████████| 5/5 [00:00<00:00, 39.73it/s, v_num=46]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params
--------------------------------------------------
0 | encoder    | Sequential         | 33.9 K
1 | attention  | MultiheadAttention | 66.0 K
2 | decoder    | Sequential         | 33.3 K
3 | classifier | Sequential         | 8.5 K 
4 | softmax    | Softmax            | 0     
--------------------------------------------------
141 K     Trainable params
0         Non-trainable params
141 K     Total params
0.567     Total estimated model params size (MB)
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `Dat

Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 62.39it/s, v_num=47]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 52.69it/s, v_num=47]
Cross-Validation Classification Report:
           0.0  1.0  2.0  accuracy  macro avg  weighted avg
precision  1.0  1.0  1.0       1.0        1.0           1.0
recall     1.0  1.0  1.0       1.0        1.0           1.0
f1-score   1.0  1.0  1.0       1.0        1.0           1.0
support    7.2  5.2  3.6       1.0       16.0          16.0


In [15]:
print(cv_results)

           0.0  1.0  2.0  accuracy  macro avg  weighted avg
precision  1.0  1.0  1.0       1.0        1.0           1.0
recall     1.0  1.0  1.0       1.0        1.0           1.0
f1-score   1.0  1.0  1.0       1.0        1.0           1.0
support    7.2  5.2  3.6       1.0       16.0          16.0


In [18]:
# Perform MC Dropout predictions
mc_predictions = mc_dropout_predictions(ae_attention_model, data_module.test_dataloader())
# Calculate mean and standard deviation for uncertainty estimates
mean_predictions = mc_predictions.mean(axis=0)
std_predictions = mc_predictions.std(axis=0)
# Convert mean predictions to class labels
predicted_labels = np.argmax(mean_predictions, axis=1)

In [19]:
# Example output with probabilities and uncertainty
for i, (mean, std) in enumerate(zip(mean_predictions, std_predictions)):
    # softmax_probs = np.exp(mean) / np.sum(np.exp(mean)) # Softmax to get probabilities
    print(f'Sample {i}: Predicted Label = {predicted_labels[i]}, Probabilities = {mean}, Uncertainty (std) = {std}')

Sample 0: Predicted Label = 2, Probabilities = [8.794023e-13 2.155976e-05 9.999785e-01], Uncertainty (std) = [4.2819121e-12 4.6762838e-05 4.6762609e-05]
Sample 1: Predicted Label = 2, Probabilities = [1.6389518e-13 1.7555063e-05 9.9998277e-01], Uncertainty (std) = [1.0847810e-12 3.2813448e-05 3.2815027e-05]
Sample 2: Predicted Label = 0, Probabilities = [9.9996918e-01 3.1090702e-05 1.9146974e-13], Uncertainty (std) = [2.4332161e-04 2.4331795e-04 1.7549427e-12]
Sample 3: Predicted Label = 0, Probabilities = [9.9999928e-01 8.5255687e-07 3.7035114e-15], Uncertainty (std) = [4.3830778e-06 4.3823616e-06 3.5958063e-14]
Sample 4: Predicted Label = 2, Probabilities = [2.5558643e-13 3.4495219e-04 9.9965525e-01], Uncertainty (std) = [1.9162801e-12 2.9945208e-03 2.9945192e-03]
Sample 5: Predicted Label = 1, Probabilities = [3.3359454e-04 9.9914080e-01 5.2574102e-04], Uncertainty (std) = [0.00134689 0.00194804 0.00125005]
Sample 6: Predicted Label = 0, Probabilities = [9.9999785e-01 2.2749416e-06 

- Export the reults

In [20]:
import pandas as pd
# Save test predictions to a CSV
test_df = pd.DataFrame({
    'Prediction': predicted_labels,
    'Probability_0': [p[0] for p in mean_predictions],
    'Probability_1': [p[1] for p in mean_predictions],
    'Probability_2': [p[2] for p in mean_predictions],  # Adjust based on num_classes
    'Uncertainty_0': [u[0] for u in std_predictions],
    'Uncertainty_1': [u[1] for u in std_predictions],
    'Uncertainty_2': [u[2] for u in std_predictions] 
})

test_df.to_csv('test_predictions.csv', index=False)

- Export the model

In [21]:
from torch import save
save(ae_attention_model, 'autoencoder_attention_classifier.pth')